In [1329]:
%reload_ext ishbook
import pandas as pd
import plus
import xlsxwriter
from IPython.core.display import display, HTML
import datetime as dt
import iql
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1330]:
teams = plus.read_google_sheet('114yDQZ7Fi6AhXBg2IhO3NZxEmAcvL12AuyXSIsZN38U',intype='key',sheet='Sheet1',headers=True,index=False)

## Collecting Team Data from Google Sheets

In [1331]:
CS_onboarding = teams[teams.Team == 'CSClientOnboarding']
CS_clientcare = teams[teams.Team == 'CSClientCare']

In [1332]:
CS_onboarding.head()

,Email,Team,Location,username
144,chante@indeed.com,CSClientOnboarding,STM,chante
172,cmammele@indeed.com,CSClientOnboarding,STM,cmammele
239,esmith@indeed.com,CSClientOnboarding,STM,esmith
388,joet@indeed.com,CSClientOnboarding,STM,joet
516,mara@indeed.com,CSClientOnboarding,STM,mara


In [1333]:
client_onboarding_team = ','.join(str(x) for x in CS_onboarding['username'])
client_care_team =','.join(str(x) for x in CS_clientcare['username'])

## Collecting Data for Chat Info for the Client Onboarding Team

In [1334]:
%%ish
chat_info = FROM crmchats 2016-10-01 2016-12-31 WHERE username IN ({client_onboarding_team}) GROUP BY username,advertiser_id, unixtime
output = None

In [1335]:
chat_info.head()

,username,advertiser_id,unixtime,
0,chante,9238976,1475478431,1
1,chante,9239014,1475480628,1
2,owen,5291697,1475484245,1
3,chante,9240264,1475484521,1
4,chante,9241158,1475486221,1


In [1336]:
chat_info.sort_values(['advertiser_id','unixtime'], ascending = True, inplace = True)
chat_info.drop_duplicates(subset = ['advertiser_id'], keep = 'first', inplace = True)

In [1337]:
chat_info.head()

,username,advertiser_id,unixtime,
144,tylerb,0,1475691675,1
1591,mara,23285,1478525276,1
3170,esmith,42571,1481197309,1
1975,pmyers,46975,1479129973,1
3004,pmyers,57650,1480954133,1


In [1338]:
chat_advertiser_info = plus.get_advertiser_info(chat_info['advertiser_id'])

In [1339]:
frd_chat = chat_advertiser_info['first_revenue_date'].to_frame()

In [1340]:
chat_info = chat_info.merge(frd_chat, how = 'left', left_on = 'advertiser_id', right_index = True)

In [1341]:
chat_info['First Chat Date This Q'] = [dt.datetime.fromtimestamp(x).strftime('%Y-%m-%d') for x in chat_info['unixtime']]

In [1342]:
len(chat_info['username'])

3426

In [1343]:
#Randomized Sample of 1000 acounts here
chat_sample_size = chat_info.sample(n=1000)

In [1344]:
chat_sample_size.head()

,username,advertiser_id,unixtime,,first_revenue_date,First Chat Date This Q
58,mwatts,9262939,1475577723,1,None,2016-10-04
89,cmammele,9268571,1475594328,1,2016-10-07,2016-10-04
1641,mara,9717281,1478540188,1,2016-11-07,2016-11-07
3172,cmammele,9906148,1481197693,1,2016-12-12,2016-12-08
1845,owen,9771916,1478853235,1,None,2016-11-11


In [1345]:
def convert_date(date):
    if date is None:
        return None
    else:
        return dt.datetime.strptime(date, '%Y-%m-%d')

In [1346]:
chat_sample_size['first_revenue_date'] = [convert_date(date) for date in chat_sample_size['first_revenue_date']]

In [1347]:
chat_sample_size['First Chat Date This Q'] = [convert_date(date) for date in chat_sample_size['First Chat Date This Q']]

In [1348]:
chat_sample_size['Difference Between FRD and First Chat'] = chat_sample_size['first_revenue_date'] - chat_sample_size['First Chat Date This Q']

In [1349]:
chat_sample_size.head(2)

,username,advertiser_id,unixtime,,first_revenue_date,First Chat Date This Q,Difference Between FRD and First Chat
58,mwatts,9262939,1475577723,1,NaT,2016-10-04,NaT
89,cmammele,9268571,1475594328,1,2016-10-07,2016-10-04,3 days


In [1350]:
condition = (chat_sample_size['Difference Between FRD and First Chat'] >= datetime.timedelta(0)) & (chat_sample_size['Difference Between FRD and First Chat'] <= datetime.timedelta(30))

In [1351]:
total_conversions_chat = len(chat_sample_size[condition]['username'])

In [1352]:
percentage_converted_from_chat = (total_conversions_chat)/1000.0

## Collecting Data for the Client Care Team

In [1353]:
client_care_team = client_care_team.replace('casey-rose,','')

In [1354]:
client_care_team

'adamk,adriana,alanat,andrewr,acillo,ariana,bkayi,candace,chaddavis,chanon,charlesf,cbarna,cangell,cjoyce,cturchioe,csatenberg,crystalf,cyrena,dcheung,dpeloso,dennisr,elena,epogosian,etrindle,elliez,faith,flato,gregoryw,hilda,jburnett,jeckelman,jmayne,jbogdanoff,joanne,jpetro,joshs,kbowman,karli,liset,lvalencia,lmerritt,marifaith,mneifert,mgray,mcardillo,mkeane,mkenny,mscimeca,micky,nlinzer,rachelw,rajvi,rcid,raymondbarba,roseannep,rbarba,ryanl,scaceres,serina,shanice,scaruso,sinclair,sdaniels,sheaslip,tgamliel,twaters,timothy,tnappi,tomgray,tschmarr,vlea,vincentp,virginia,whitneys,yuliya,zehra,snicole,tmichaud'

In [1398]:
phone_record = pd.read_csv("calls_list.csv")[["caller_number", "caller_name", "g_name", "c_name",
                                             "call_s",'duration' ,"call_status",]]

In [1399]:
phone_record["caller_number"] = phone_record["caller_number"].str.replace(r'[^0-9]+', '')

In [1400]:
phone_record.head()

,caller_number,caller_name,g_name,c_name,call_s,duration,call_status
0,8606358747,Cromwell CT,Sandcrawler,dradisfunnel: en_US,2016-12-31 13:37:36,00:00:28,ANSWER
1,2032540545,DRCUEVAS,Sandcrawler,dradisfunnel: en_US,2016-12-31 09:15:42,00:01:15,ANSWER
2,9373391255,PATTERSON & COM,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:56:54,00:05:40,ANSWER
3,9725133932,IRVING TX,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:24:50,00:42:07,ANSWER
4,6199971321,WIRELESS CALLER,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:14:03,00:07:31,ANSWER


In [1358]:
phone_record.drop_duplicates(subset = ['caller_name'], keep = 'last', inplace = True)

In [1359]:
%%ish
call_info = FROM crmcalls 2016-10-01 2016-12-31 WHERE rep_username IN ({client_care_team}) client_id != 0 GROUP BY rep_username,client_id, external_number, talk_time_seconds, unixtime
output = None

In [1360]:
call_info["external_number"] = call_info["external_number"].astype("str")

In [1361]:
call_info["external_number"] = call_info["external_number"].str.replace(r'[^0-9]+', '')

In [1362]:
def remove_areacode(s):
    return s[1:]

In [1363]:
call_info['external_number'] = [remove_areacode(s) for s in call_info['external_number']]

In [1364]:
call_info.sort_values('client_id').head()

,rep_username,client_id,external_number,talk_time_seconds,unixtime,
32322,whitneys,23,2037501030,191,1481657376,1
3838,adriana,154,2014890256,374,1476106122,1
34401,ariana,438,2125575000,307,1482170707,1
32796,jbogdanoff,438,2125575000,480,1481735858,1
8369,rbarba,475,4258828080,277,1476892760,1


In [1365]:
call_funnel_data = phone_record.merge(call_info,how = 'left', left_on = 'caller_number', right_on = 'external_number')
call_funnel_data.dropna(inplace = True)
call_funnel_data.head()

,caller_number,caller_name,g_name,c_name,call_s,duration,call_status,rep_username,client_id,external_number,talk_time_seconds,unixtime,
1,9373391255,PATTERSON & COM,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:56:54,00:05:40,ANSWER,lmerritt,10323798.0,9373391255,303.0,1.483135e+09,1.0
2,9725133932,IRVING TX,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:11:05,00:04:02,ANSWER,whitneys,10323742.0,9725133932,194.0,1.483132e+09,1.0
3,9725133932,IRVING TX,Sandcrawler,dradisfunnel: en_US,2016-12-30 16:11:05,00:04:02,ANSWER,ryanl,10323742.0,9725133932,2352.0,1.483133e+09,1.0
6,4192140195,H4T AUTOS,Sandcrawler,dradisfunnel: en_US,2016-12-30 14:56:17,00:07:15,ANSWER,scaceres,10316878.0,4192140195,356.0,1.483128e+09,1.0
7,9415484474,ENGLEWOOD FL,Sandcrawler,dradisfunnel: en_US,2016-12-30 14:34:56,00:04:39,ANSWER,adriana,10115938.0,9415484474,242.0,1.483127e+09,1.0


In [1366]:
call_advertiser_info = plus.get_advertiser_info(call_funnel_data['client_id'])

In [1367]:
frd_call = call_advertiser_info['first_revenue_date'].to_frame()

In [1368]:
call_funnel_data = call_funnel_data.merge(frd_call, how = 'left', left_on = 'client_id', right_index = True)

In [1369]:
#Randomize Here
call_sample_size = call_funnel_data.sample(n=1000)

In [1370]:
call_sample_size.head(2)

,caller_number,caller_name,g_name,c_name,call_s,duration,call_status,rep_username,client_id,external_number,talk_time_seconds,unixtime,,first_revenue_date
499,2159684236,NEWTOWN PA,Sandcrawler,dradisfunnel: en_US,2016-12-05 11:44:15,00:16:38,ANSWER,jburnett,10067930.0,2159684236,640.0,1.481139e+09,1.0,None
1005,3303435812,BERNER TRUCKING,Sandcrawler,dradisfunnel: en_US,2016-11-10 13:59:04,00:02:44,ANSWER,timothy,9748214.0,3303435812,326.0,1.481552e+09,1.0,2016-12-12


In [1371]:
call_sample_size['first_revenue_date'] = [convert_date(date) for date in call_sample_size['first_revenue_date']]

In [1372]:
def remove_unwanted(s):
    return s[0:10]

In [1373]:
call_sample_size['call_s'] = [remove_unwanted(s) for s in call_sample_size['call_s']]

In [1374]:
call_sample_size['call_s'] = [convert_date(date) for date in call_sample_size['call_s']]

In [1375]:
call_sample_size['Difference Between FRD and First Call'] = call_sample_size['first_revenue_date'] - call_sample_size['call_s']

In [1376]:
call_sample_size.head()

,caller_number,caller_name,g_name,c_name,call_s,duration,call_status,rep_username,client_id,external_number,talk_time_seconds,unixtime,,first_revenue_date,Difference Between FRD and First Call
499,2159684236,NEWTOWN PA,Sandcrawler,dradisfunnel: en_US,2016-12-05,00:16:38,ANSWER,jburnett,10067930.0,2159684236,640.0,1.481139e+09,1.0,NaT,NaT
1005,3303435812,BERNER TRUCKING,Sandcrawler,dradisfunnel: en_US,2016-11-10,00:02:44,ANSWER,timothy,9748214.0,3303435812,326.0,1.481552e+09,1.0,2016-12-12,32 days
2054,7607058888,GOOGLE GUY,Sandcrawler,dradisfunnel: en_US,2016-10-11,00:00:48,ANSWER,elliez,122082.0,7607058888,287.0,1.482856e+09,1.0,NaT,NaT
1761,6195460621,POTHOS INC,Sandcrawler,dradisfunnel: en_US,2016-10-18,00:10:32,ANSWER,sheaslip,1555878.0,6195460621,551.0,1.476819e+09,1.0,NaT,NaT
833,3037593999,DRAIN GENIE INC,Sandcrawler,dradisfunnel: en_US,2016-11-16,00:05:51,ANSWER,shanice,9836897.0,3037593999,332.0,1.479326e+09,1.0,2016-11-16,0 days


In [1377]:
condition = (call_sample_size['Difference Between FRD and First Call'] >= datetime.timedelta(0)) & (call_sample_size['Difference Between FRD and First Call'] <= datetime.timedelta(30))

In [1378]:
total_conversions_phone = len(call_sample_size[condition]['client_id'])

In [1379]:
percentage_converted_from_phone = total_conversions_phone/1000.0

### Collecting Revenue Data for Client Care Team

In [1380]:
phone_revenue_data = pd.read_csv("converted_accounts_phone.csv")

In [1381]:
test = call_sample_size.merge(phone_revenue_data, how = 'left', left_on = 'client_id', right_on = 'Advertiser ID')

In [1382]:
total_revenue_phone = test['Revenue (USD)'].sum()

In [1383]:
total_revenue_phone

22797.12

### Collecting Revenue Data for Client Onboarding

In [1384]:
chat_revenue_data = pd.read_csv("converted_accounts_chat.csv")

In [1385]:
test2 = chat_sample_size.merge(chat_revenue_data, how = 'left', left_on = 'advertiser_id', right_on = 'Advertiser ID')

In [1386]:
total_revenue_chat = test2['Revenue (USD)'].sum()

In [1387]:
total_revenue_chat

33613.38999999999

## FINAL DATA RESULTS FOR CHAT

In [1388]:
total_conversions_chat

213

In [1389]:
percentage_converted_from_chat

0.213

In [1390]:
total_revenue_chat

33613.38999999999

## FINAL DATA RESULTS FOR PHONE

In [1391]:
total_conversions_phone

170

In [1392]:
percentage_converted_from_phone

0.17

In [1393]:
total_revenue_phone

22797.12

In [1394]:
#test.drop({'g_name','duration','call_status','external_number','talk_time_seconds','unixtime',''},1)

In [1395]:
test.to_csv('Phone Info.csv', encoding = 'utf-8')

In [1396]:
test2.to_csv("Chat Info.csv", encoding = 'utf-8')